**Chapter 14 – Deep Computer Vision Using Convolutional Neural Networks**

_This notebook contains all the sample code in chapter 14._

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/ageron/handson-ml2/blob/master/14_deep_computer_vision_with_cnns.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [ ]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, losses

assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


A couple utility functions to plot grayscale and RGB images:

In [ ]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

# Tackling Fashion MNIST With a CNN

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:]
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

X_mean = X_train.mean(axis=0, keepdims=True)
X_std = X_train.std(axis=0, keepdims=True) + 1e-7
X_train = (X_train - X_mean) / X_std
X_valid = (X_valid - X_mean) / X_std
X_test = (X_test - X_mean) / X_std

X_train = X_train[..., np.newaxis]
X_valid = X_valid[..., np.newaxis]
X_test = X_test[..., np.newaxis]

In [ ]:
numhiddens=2
num_filters=32
num_epochs=10

DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu', padding="SAME")
model = tf.keras.Sequential()
model.add(DefaultConv2D(filters=num_filters, kernel_size=5, input_shape=[28, 28, 1]))
for i in range(numhiddens):
    model.add(DefaultConv2D(filters=num_filters))
    if(i%2==0):
      num_filters=num_filters*2  
      model.add(keras.layers.MaxPooling2D(pool_size=2))

model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(units=10, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_5 (Conv2D)           (None, 28, 28, 32)        832       
                                                                 
 conv2d_6 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 flatten_1 (Flatten)         (None, 12544)             0         
                                                                 
 dense_3 (Dense)             (None, 64)                802880    
                                                      

In [ ]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_valid, y_valid))
score = model.evaluate(X_test, y_test)


Epoch 1/10
1719/1719 [==============================] - 16s 7ms/step - loss: 0.7337 - accuracy: 0.7357 - val_loss: 0.4201 - val_accuracy: 0.8406
Epoch 2/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4920 - accuracy: 0.8266 - val_loss: 0.3658 - val_accuracy: 0.8614
Epoch 3/10
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4283 - accuracy: 0.8478 - val_loss: 0.3328 - val_accuracy: 0.8750
Epoch 4/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3947 - accuracy: 0.8609 - val_loss: 0.3155 - val_accuracy: 0.8780
Epoch 5/10
1719/1719 [==============================] - 8s 5ms/step - loss: 0.3658 - accuracy: 0.8702 - val_loss: 0.2963 - val_accuracy: 0.8868
Epoch 6/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3450 - accuracy: 0.8783 - val_loss: 0.2935 - val_accuracy: 0.8896
Epoch 7/10
1719/1719 [==============================] - 7s 4ms/step - loss: 0.3242 - accuracy: 0.8849 - val_loss: 0.2730 - val_accuracy

Next, compare how the model performs on the test dataset:

In [ ]:
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

print('\nTest accuracy:', test_acc)

313/313 - 1s - loss: 0.2877 - accuracy: 0.8957 - 742ms/epoch - 2ms/step

Test accuracy: 0.8956999778747559


Prediction of a single image

In [ ]:
X_new = X_test[:10] # pretend we have new images
y_pred = model.predict(X_new)
y_pred

1/1 [==============================] - 0s 141ms/step


array([[2.71345169e-09, 1.70923951e-08, 6.05659967e-09, 6.98474523e-09,
        2.00400851e-09, 3.12830634e-05, 3.24559650e-08, 2.09485251e-03,
        2.83858725e-07, 9.97873545e-01],
       [3.55425691e-05, 3.97341576e-10, 9.98585939e-01, 4.57513011e-07,
        7.84197531e-04, 1.65624930e-10, 5.93666453e-04, 3.12760061e-12,
        8.46997850e-08, 1.29134490e-11],
       [2.08135766e-08, 9.99999881e-01, 6.74223871e-11, 7.66416761e-08,
        1.51976813e-08, 1.13624279e-12, 1.09951932e-08, 3.09670345e-12,
        2.17374729e-12, 6.89775320e-12],
       [2.95047098e-10, 9.99999642e-01, 4.89828447e-13, 3.63504910e-07,
        6.58521460e-10, 5.89304809e-14, 6.65922803e-11, 8.63975368e-14,
        1.30031470e-13, 5.72884839e-13],
       [2.33706962e-02, 5.74157821e-06, 5.41824149e-03, 2.89824442e-03,
        4.12345305e-03, 1.45684375e-04, 9.63959754e-01, 6.50849279e-06,
        4.73884429e-05, 2.43310533e-05],
       [9.70020778e-07, 9.99991298e-01, 4.37668817e-08, 5.53969039e-06,
   

#Homework 5: High Accuracy CNN for Fashion MNIST
_Exercise: Build your own CNN from scratch and try to achieve the highest possible accuracy on Fashion MNIST._

Use can consider different architectures, BatchNormalization, LayerNormalization, Dropout, SkipConnections, data augmentation techniques, etc.

Moreover you can consider to use Grid search or Randomized search or [Optuna](https://colab.research.google.com/github/serivan/mldmlab/blob/master/Solutions/Optuna_bayesian_hyperparameter_tuning.ipynb) in order to  define the hyperparameters.